# Brain Tumor Classification - Model Comparison

This notebook compares the baseline model explored in the previous section with more advanced models including using residual connections, and using Vision Transformers.

## 1. Setup and Imports

In [1]:
#!/usr/bin/env python
# coding: utf-8
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
import matplotlib.pyplot as plt
import timm
import time
from torchsummary import summary

In [2]:
# Topology imports

!pip install gudhi --quiet
!pip install PersistenceImages --quiet
import gudhi as gd
import PersistenceImages.persistence_images as pimg

In [3]:
# # import our own modules (if run locally)
# import sys
# sys.path.append('../src')  # Add the src directory to the Python path

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Use os.path.join to avoid manual path issues
#path = os.path.join('gdrive', 'My Drive', 'Erdos', 'brainnet-medical-imaging', 'src')
os.chdir('/content/drive/My Drive/Deep Learning/BrainNet-Medical Imaging/src')

# Change directory
# os.getcwd()
#os.chdir(path)

Mounted at /content/drive


In [4]:
# import data
from config.data import data_setup, data_loader, add_topo_features
from models.vision_transformer import run_training
from utils.prediction import analyze_predictions, train_model
from models.cnn import BrainTumorCNN, BrainTumorCNN_RN, BrainTumorCNN_Topo
from utils.visualization import plot_epoch_times, plot_accuracy_results

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda
Using device: cuda
Using device: cuda


## 2. Data Loading and Preprocessing

In [5]:
# Load data and data_loaders
train_set, test_set, label_conversion_dict = data_setup()
train_loader, test_loader = data_loader(train_set, test_set)

# Extract the class labels from the dictionary keys
class_labels = list(label_conversion_dict.keys())[:4]

/content/drive/My Drive/Deep Learning/BrainNet-Medical Imaging/src/config/data.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['label'] = df_unique['filepath'].apply(lambda x: x.split('/')[-2])
/content/drive/My Drive/Deep Learning/BrainNet-Medical Imaging/src/config/data.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['class'] = df_unique['filename'].apply(lambda x: 'train' if x[:2] == 'Tr' else 'test')
100%|██████████| 6726/6726 [00:03<00:00, 1705.83it/s]


Initial number of samples in train set: 5521
Initial number of samples in test set: 1205
Total samples in train set after overriding labels: 5521
Total samples in test set after overriding labels: 1205
Total files in train set: 5521, with target values: [0, 1, 2, 3]
Total files in test set: 1205, with target values: [0, 1, 2, 3]


# 3. Model training

We start with our baseline model

In [6]:
# Initialize model
model_cnn = BrainTumorCNN().to(device)
print(model_cnn)

BrainTumorCNN(
  (features): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.01)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.01)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [7]:
history = train_model(
    model=model_cnn,
    train_loader=train_loader,
    val_loader=test_loader,
    num_epochs=10
)

# Now you can access the following:
cnn_train_loss_list = history['train_loss_list']
cnn_test_loss_list = history['test_loss_list']
cnn_accuracy_list = history['accuracy_list']
cnn_all_predictions = history['all_predictions']
cnn_all_true_labels = history['all_true_labels']

Epoch 1/10:
Train Loss: 0.7075, Val Loss: 0.5481, Accuracy: 76.60%
Epoch 1 took 9.15 seconds
Epoch 2/10:
Train Loss: 0.4219, Val Loss: 0.3796, Accuracy: 84.81%
Epoch 2 took 8.81 seconds
Epoch 3/10:
Train Loss: 0.3339, Val Loss: 0.3690, Accuracy: 85.39%
Epoch 3 took 8.71 seconds
Epoch 4/10:
Train Loss: 0.2617, Val Loss: 0.3239, Accuracy: 86.56%
Epoch 4 took 8.76 seconds
Epoch 5/10:
Train Loss: 0.2084, Val Loss: 0.3074, Accuracy: 89.13%
Epoch 5 took 8.77 seconds
Epoch 6/10:
Train Loss: 0.1658, Val Loss: 0.3132, Accuracy: 87.80%
Epoch 6 took 8.68 seconds
Epoch 7/10:
Train Loss: 0.1536, Val Loss: 0.1837, Accuracy: 93.03%
Epoch 7 took 8.62 seconds
Epoch 8/10:
Train Loss: 0.0999, Val Loss: 0.2065, Accuracy: 92.12%
Epoch 8 took 8.72 seconds
Epoch 9/10:
Train Loss: 0.1096, Val Loss: 0.5257, Accuracy: 80.91%
Epoch 9 took 8.72 seconds
Epoch 10/10:
Train Loss: 0.0946, Val Loss: 0.2034, Accuracy: 93.44%
Epoch 10 took 8.66 seconds


In [8]:
# Print the mean of the epoch times
cnn_epoch_times = history['epoch_times']
print(f"CNN mean epoch time: {np.mean(cnn_epoch_times):.2f} seconds")

CNN mean epoch time: 8.76 seconds


In [9]:
# Analyse and save predictions
analyze_predictions(cnn_all_predictions, cnn_all_true_labels)

# Create a directory called "results" if it doesn't already exist
os.makedirs('results', exist_ok=True)

# Save the lists as numpy arrays for easy reloading
np.save('results/cnn_train_loss_list.npy', np.array(cnn_train_loss_list))
np.save('results/cnn_test_loss_list.npy', np.array(cnn_test_loss_list))
np.save('results/cnn_accuracy_list.npy', np.array(cnn_accuracy_list))
np.save('results/cnn_epoch_times_list.npy', np.array(cnn_epoch_times))

# Save predictions and true labels as pickle files (more efficient for storing lists of lists)
with open('results/cnn_all_predictions.pkl', 'wb') as f:
    pickle.dump(cnn_all_predictions, f)

with open('results/cnn_all_true_labels.pkl', 'wb') as f:
    pickle.dump(cnn_all_true_labels, f)

print("All results have been successfully saved in the 'results/' directory.")


Epoch 1 Prediction Counts: Counter({np.int64(3): 380, np.int64(0): 303, np.int64(1): 276, np.int64(2): 246})
Epoch 2 Prediction Counts: Counter({np.int64(2): 338, np.int64(1): 310, np.int64(3): 293, np.int64(0): 264})
Epoch 3 Prediction Counts: Counter({np.int64(2): 369, np.int64(1): 317, np.int64(3): 272, np.int64(0): 247})
Epoch 4 Prediction Counts: Counter({np.int64(1): 372, np.int64(2): 336, np.int64(3): 294, np.int64(0): 203})
Epoch 5 Prediction Counts: Counter({np.int64(3): 347, np.int64(2): 334, np.int64(1): 268, np.int64(0): 256})
Epoch 6 Prediction Counts: Counter({np.int64(2): 354, np.int64(1): 327, np.int64(3): 276, np.int64(0): 248})
Epoch 7 Prediction Counts: Counter({np.int64(2): 335, np.int64(3): 312, np.int64(0): 281, np.int64(1): 277})
Epoch 8 Prediction Counts: Counter({np.int64(1): 337, np.int64(2): 330, np.int64(3): 304, np.int64(0): 234})
Epoch 9 Prediction Counts: Counter({np.int64(0): 397, np.int64(2): 324, np.int64(1): 291, np.int64(3): 193})
Epoch 10 Prediction

Now we try the same CNN but with residual connections.

In [10]:
model_cnn_res = BrainTumorCNN_RN().to(device)
print(model_cnn_res)

BrainTumorCNN_RN(
  (features): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.01)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): ResidualBlock(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0)

In [11]:
history = train_model(
    model=model_cnn_res,
    train_loader=train_loader,
    val_loader=test_loader,
    num_epochs=10
)

Epoch 1/10:
Train Loss: 0.8822, Val Loss: 0.5412, Accuracy: 77.26%
Epoch 1 took 8.76 seconds
Epoch 2/10:
Train Loss: 0.4153, Val Loss: 0.5060, Accuracy: 79.75%
Epoch 2 took 8.78 seconds
Epoch 3/10:
Train Loss: 0.3370, Val Loss: 0.4179, Accuracy: 82.90%
Epoch 3 took 8.83 seconds
Epoch 4/10:
Train Loss: 0.2902, Val Loss: 0.3450, Accuracy: 86.14%
Epoch 4 took 8.69 seconds
Epoch 5/10:
Train Loss: 0.2516, Val Loss: 0.4879, Accuracy: 82.16%
Epoch 5 took 8.84 seconds
Epoch 6/10:
Train Loss: 0.2125, Val Loss: 0.3687, Accuracy: 86.56%
Epoch 6 took 8.86 seconds
Epoch 7/10:
Train Loss: 0.1923, Val Loss: 0.2766, Accuracy: 87.80%
Epoch 7 took 8.81 seconds
Epoch 8/10:
Train Loss: 0.1801, Val Loss: 0.3020, Accuracy: 87.55%
Epoch 8 took 8.72 seconds
Epoch 9/10:
Train Loss: 0.1596, Val Loss: 0.1578, Accuracy: 93.78%
Epoch 9 took 8.75 seconds
Epoch 10/10:
Train Loss: 0.1141, Val Loss: 0.1421, Accuracy: 95.52%
Epoch 10 took 8.74 seconds


In [12]:
# Now you can access the following:
cnn_res_train_loss_list = history['train_loss_list']
cnn_res_test_loss_list = history['test_loss_list']
cnn_res_accuracy_list = history['accuracy_list']
cnn_res_all_predictions = history['all_predictions']
cnn_res_all_true_labels = history['all_true_labels']

# Print the mean of the epoch times
cnn_res_epoch_times = history['epoch_times']
print(f"CNN (Res) mean epoch time: {np.mean(cnn_res_epoch_times):.2f} seconds")

CNN (Res) mean epoch time: 8.78 seconds


In [13]:
# Analyse and save predictions
analyze_predictions(cnn_res_all_predictions, cnn_res_all_true_labels)

# Create a directory called "results" if it doesn't already exist
os.makedirs('results', exist_ok=True)

# Save the lists as numpy arrays for easy reloading
np.save('results/cnn_res_train_loss_list.npy', np.array(cnn_res_train_loss_list))
np.save('results/cnn_res_test_loss_list.npy', np.array(cnn_res_test_loss_list))
np.save('results/cnn_res_accuracy_list.npy', np.array(cnn_res_accuracy_list))
np.save('results/cnn_res_epoch_times_list.npy', np.array(cnn_res_epoch_times))

# Save predictions and true labels as pickle files (more efficient for storing lists of lists)
with open('results/cnn_res_all_predictions.pkl', 'wb') as f:
    pickle.dump(cnn_res_all_predictions, f)

with open('results/cnn_res_all_true_labels.pkl', 'wb') as f:
    pickle.dump(cnn_res_all_true_labels, f)

print("All results have been successfully saved in the 'results/' directory.")


Epoch 1 Prediction Counts: Counter({np.int64(2): 386, np.int64(3): 288, np.int64(0): 277, np.int64(1): 254})
Epoch 2 Prediction Counts: Counter({np.int64(2): 417, np.int64(3): 328, np.int64(0): 281, np.int64(1): 179})
Epoch 3 Prediction Counts: Counter({np.int64(2): 389, np.int64(0): 316, np.int64(3): 277, np.int64(1): 223})
Epoch 4 Prediction Counts: Counter({np.int64(3): 349, np.int64(2): 326, np.int64(1): 305, np.int64(0): 225})
Epoch 5 Prediction Counts: Counter({np.int64(1): 450, np.int64(2): 310, np.int64(3): 306, np.int64(0): 139})
Epoch 6 Prediction Counts: Counter({np.int64(2): 394, np.int64(0): 330, np.int64(3): 288, np.int64(1): 193})
Epoch 7 Prediction Counts: Counter({np.int64(2): 384, np.int64(3): 291, np.int64(1): 270, np.int64(0): 260})
Epoch 8 Prediction Counts: Counter({np.int64(2): 364, np.int64(3): 327, np.int64(1): 297, np.int64(0): 217})
Epoch 9 Prediction Counts: Counter({np.int64(2): 330, np.int64(1): 301, np.int64(3): 295, np.int64(0): 279})
Epoch 10 Prediction

Now we try the same CNN but with topological features.

In [14]:
model_cnn_topo = BrainTumorCNN_Topo().to(device)
print(model_cnn_topo)

BrainTumorCNN_Topo(
  (cnv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (maxpool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (leakyRelu): LeakyReLU(negative_slope=0.01)
  (fc1): Linear(in_features=2073, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=4, bias=True)
)


In [15]:
train_set_topo, test_set_topo = add_topo_features(train_set,test_set) # takes a while to load topo data - around 10 minutes
train_loader_topo, test_loader_topo = data_loader(train_set_topo, test_set_topo)

In [16]:
history = train_model(
    model=model_cnn_topo,
    train_loader=train_loader_topo,
    val_loader=test_loader_topo,
    num_epochs=10
)

Epoch 1/10:
Train Loss: 0.9825, Val Loss: 0.7957, Accuracy: 67.97%
Epoch 1 took 9.04 seconds
Epoch 2/10:
Train Loss: 0.5507, Val Loss: 0.5631, Accuracy: 78.76%
Epoch 2 took 8.85 seconds
Epoch 3/10:
Train Loss: 0.3520, Val Loss: 0.3938, Accuracy: 84.15%
Epoch 3 took 8.82 seconds
Epoch 4/10:
Train Loss: 0.2608, Val Loss: 0.2757, Accuracy: 88.96%
Epoch 4 took 8.94 seconds
Epoch 5/10:
Train Loss: 0.1813, Val Loss: 0.3413, Accuracy: 87.05%
Epoch 5 took 8.98 seconds
Epoch 6/10:
Train Loss: 0.1505, Val Loss: 0.1727, Accuracy: 94.19%
Epoch 6 took 8.81 seconds
Epoch 7/10:
Train Loss: 0.0975, Val Loss: 0.2728, Accuracy: 90.62%
Epoch 7 took 8.92 seconds
Epoch 8/10:
Train Loss: 0.0812, Val Loss: 0.1349, Accuracy: 95.19%
Epoch 8 took 8.95 seconds
Epoch 9/10:
Train Loss: 0.0641, Val Loss: 0.1649, Accuracy: 94.85%
Epoch 9 took 8.97 seconds
Epoch 10/10:
Train Loss: 0.0421, Val Loss: 0.1468, Accuracy: 95.44%
Epoch 10 took 8.85 seconds


In [17]:
# Now you can access the following:
cnn_topo_train_loss_list = history['train_loss_list']
cnn_topo_test_loss_list = history['test_loss_list']
cnn_topo_accuracy_list = history['accuracy_list']
cnn_topo_all_predictions = history['all_predictions']
cnn_topo_all_true_labels = history['all_true_labels']

# Print the mean of the epoch times
cnn_topo_epoch_times = history['epoch_times']
print(f"CNN (Topo) mean epoch time: {np.mean(cnn_topo_epoch_times):.2f} seconds")

CNN (Topo) mean epoch time: 8.91 seconds


In [18]:
# Analyse and save predictions
analyze_predictions(cnn_topo_all_predictions, cnn_topo_all_true_labels)

# Create a directory called "results" if it doesn't already exist
os.makedirs('results', exist_ok=True)

# Save the lists as numpy arrays for easy reloading
np.save('results/cnn_topo_train_loss_list.npy', np.array(cnn_topo_train_loss_list))
np.save('results/cnn_topo_test_loss_list.npy', np.array(cnn_topo_test_loss_list))
np.save('results/cnn_topo_accuracy_list.npy', np.array(cnn_topo_accuracy_list))
np.save('results/cnn_topo_epoch_times_list.npy', np.array(cnn_topo_epoch_times))

# Save predictions and true labels as pickle files (more efficient for storing lists of lists)
with open('results/cnn_topo_all_predictions.pkl', 'wb') as f:
    pickle.dump(cnn_topo_all_predictions, f)

with open('results/cnn_topo_all_true_labels.pkl', 'wb') as f:
    pickle.dump(cnn_topo_all_true_labels, f)

print("All results have been successfully saved in the 'results/' directory.")

Epoch 1 Prediction Counts: Counter({np.int64(3): 396, np.int64(2): 376, np.int64(0): 238, np.int64(1): 195})
Epoch 2 Prediction Counts: Counter({np.int64(2): 372, np.int64(3): 331, np.int64(0): 259, np.int64(1): 243})
Epoch 3 Prediction Counts: Counter({np.int64(1): 366, np.int64(2): 321, np.int64(3): 306, np.int64(0): 212})
Epoch 4 Prediction Counts: Counter({np.int64(2): 328, np.int64(0): 326, np.int64(3): 312, np.int64(1): 239})
Epoch 5 Prediction Counts: Counter({np.int64(0): 389, np.int64(3): 302, np.int64(2): 295, np.int64(1): 219})
Epoch 6 Prediction Counts: Counter({np.int64(2): 315, np.int64(1): 313, np.int64(3): 293, np.int64(0): 284})
Epoch 7 Prediction Counts: Counter({np.int64(1): 340, np.int64(3): 321, np.int64(2): 313, np.int64(0): 231})
Epoch 8 Prediction Counts: Counter({np.int64(1): 314, np.int64(2): 302, np.int64(3): 300, np.int64(0): 289})
Epoch 9 Prediction Counts: Counter({np.int64(1): 314, np.int64(2): 311, np.int64(3): 304, np.int64(0): 276})
Epoch 10 Prediction

We use a pre-trained ViT, else we would overfit the data since ViT need a lot of data and we only have a few thousand images

In [19]:
# Load a pre-trained Vision Transformer model
vit_model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=4)  # Adjust 'num_classes' to your task
vit_model.to(device)
vit_model.train()  # Set model to training mode

# Print the model architecture
verbose = 0
if verbose:
  summary(vit_model, input_size=(3, 224, 224))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [20]:
train_set_vit, test_set_vit, label_conversion_dict = data_setup(vision_transformer=True)
train_loader_vit, test_loader_vit = data_loader(train_set_vit, test_set_vit)

/content/drive/My Drive/Deep Learning/BrainNet-Medical Imaging/src/config/data.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['label'] = df_unique['filepath'].apply(lambda x: x.split('/')[-2])
/content/drive/My Drive/Deep Learning/BrainNet-Medical Imaging/src/config/data.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['class'] = df_unique['filename'].apply(lambda x: 'train' if x[:2] == 'Tr' else 'test')
100%|██████████| 6726/6726 [00:03<00:00, 1743.19it/s]


Initial number of samples in train set: 5521
Initial number of samples in test set: 1205
Total samples in train set after overriding labels: 5521
Total samples in test set after overriding labels: 1205
Total files in train set: 5521, with target values: [0, 1, 2, 3]
Total files in test set: 1205, with target values: [0, 1, 2, 3]


In [21]:
# Standard optimizer setup
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters(), lr=0.001)

# Set number of epochs
num_epochs = 10

# Run training
train_loss_list, test_loss_list, accuracy_list, vit_all_predictions, vit_all_true_labels, vit_epoch_times = run_training(
    vit_model, train_loader_vit, test_loader_vit, optimizer, None, criterion, num_epochs, device, fine_tuning=False
)

# Print the mean epoch time
vit_mean_epoch_time = np.mean(vit_epoch_times)
print(f"ViT mean epoch time: {vit_mean_epoch_time:.2f} seconds")

Epoch [1/10], Train Loss: 1.5703, Val Loss: 1.0820, Accuracy: 51.87%, Epoch Time: 171.97 seconds
Epoch [2/10], Train Loss: 1.0151, Val Loss: 0.9423, Accuracy: 58.76%, Epoch Time: 177.05 seconds
Epoch [3/10], Train Loss: 0.9087, Val Loss: 1.1178, Accuracy: 56.51%, Epoch Time: 178.25 seconds
Epoch [4/10], Train Loss: 0.7968, Val Loss: 1.1164, Accuracy: 53.86%, Epoch Time: 178.26 seconds
Epoch [5/10], Train Loss: 0.7028, Val Loss: 0.6636, Accuracy: 71.62%, Epoch Time: 178.62 seconds
Epoch [6/10], Train Loss: 0.5897, Val Loss: 0.7281, Accuracy: 69.46%, Epoch Time: 178.49 seconds
Epoch [7/10], Train Loss: 0.5570, Val Loss: 0.6280, Accuracy: 74.61%, Epoch Time: 178.58 seconds
Epoch [8/10], Train Loss: 0.5502, Val Loss: 0.6466, Accuracy: 72.70%, Epoch Time: 178.51 seconds
Epoch [9/10], Train Loss: 0.4626, Val Loss: 0.5368, Accuracy: 77.10%, Epoch Time: 178.52 seconds
Epoch [10/10], Train Loss: 0.4527, Val Loss: 0.6069, Accuracy: 78.34%, Epoch Time: 178.58 seconds
ViT mean epoch time: 177.68 s

In [22]:
# Analyse and save predictions
analyze_predictions(vit_all_predictions, vit_all_true_labels)

# Create a directory called "results" if it doesn't already exist
os.makedirs('results', exist_ok=True)

# Save the lists as numpy arrays for easy reloading
np.save('results/vit_train_loss_list.npy', np.array(train_loss_list))
np.save('results/vit_test_loss_list.npy', np.array(test_loss_list))
np.save('results/vit_accuracy_list.npy', np.array(accuracy_list))
np.save('results/vit_epoch_times_list.npy', np.array(vit_epoch_times))

# Save predictions and true labels as pickle files (more efficient for storing lists of lists)
with open('results/vit_all_predictions.pkl', 'wb') as f:
    pickle.dump(vit_all_predictions, f)

with open('results/vit_all_true_labels.pkl', 'wb') as f:
    pickle.dump(vit_all_true_labels, f)

print("All results have been successfully saved in the 'results/' directory.")



Epoch 1 Prediction Counts: Counter({np.int64(3): 598, np.int64(2): 435, np.int64(0): 169, np.int64(1): 3})
Epoch 2 Prediction Counts: Counter({np.int64(1): 433, np.int64(2): 339, np.int64(0): 250, np.int64(3): 183})
Epoch 3 Prediction Counts: Counter({np.int64(0): 418, np.int64(2): 381, np.int64(1): 330, np.int64(3): 76})
Epoch 4 Prediction Counts: Counter({np.int64(0): 721, np.int64(2): 278, np.int64(3): 106, np.int64(1): 100})
Epoch 5 Prediction Counts: Counter({np.int64(1): 454, np.int64(2): 287, np.int64(0): 245, np.int64(3): 219})
Epoch 6 Prediction Counts: Counter({np.int64(0): 316, np.int64(1): 316, np.int64(3): 299, np.int64(2): 274})
Epoch 7 Prediction Counts: Counter({np.int64(2): 460, np.int64(3): 284, np.int64(0): 264, np.int64(1): 197})
Epoch 8 Prediction Counts: Counter({np.int64(2): 387, np.int64(3): 330, np.int64(1): 310, np.int64(0): 178})
Epoch 9 Prediction Counts: Counter({np.int64(3): 386, np.int64(2): 313, np.int64(1): 285, np.int64(0): 221})
Epoch 10 Prediction Co

### Now we explore a second version of the ViT model where we unfreeze the last Transformation block.

Key extras about this model
 - Pre-trained Model: Like the first model, this one starts with a pre-trained Vision Transformer, but only the last transformer block and the classification head are unfrozen.
 - Fine-tuning: The last transformer block (blocks.11) and the normalization layers are unfrozen. This allows the model to adjust the more complex representations in the deeper layers of the transformer to fit your specific task.
 - (There are 12 transformer blocks in total)

In [23]:
# # Load a pre-trained Vision Transformer model
vit_model_ft = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=4)  # Adjust 'num_classes' to your task

# Fine-tuning setup (assuming you want to unfreeze specific layers)
for name, param in vit_model_ft.named_parameters():
    if 'blocks.11' in name or 'norm' in name or 'head' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Add Dropout layer before final classification layer
vit_model_ft.head = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(vit_model_ft.head.in_features, 4)
)

vit_model_ft.to(device)
vit_model_ft.train()

# Print the model architecture
verbose = 0
if verbose:
  summary(vit_model_ft, input_size=(3, 224, 224))

In [24]:
# Optimizer setup with different learning rates for head and other layers
head_params = list(vit_model_ft.head.parameters())
other_params = [p for p in vit_model_ft.parameters() if p not in set(head_params)]

optimizer = Adam([
    {'params': head_params, 'lr': 0.001},
    {'params': other_params, 'lr': 0.0001}
])

# Scheduler setup if needed
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [25]:
# Run training
vit_ft_train_loss_list, vit_ft_test_loss_list, vit_ft_accuracy_list, vit_ft_all_predictions, vit_ft_all_true_labels, vit_ft_epoch_times = run_training(
    vit_model_ft, train_loader_vit, test_loader_vit, optimizer, scheduler, criterion, num_epochs, device, fine_tuning=True
)

# Print the mean epoch time
vit_ft_mean_epoch_time = np.mean(vit_ft_epoch_times)
print(f"ViT (fine-tuned) epoch time: {vit_ft_mean_epoch_time:.2f} seconds")

Epoch [1/10], Train Loss: 0.5354, Val Loss: 0.2955, Accuracy: 87.72%, Epoch Time: 138.76 seconds
Epoch [2/10], Train Loss: 0.1832, Val Loss: 0.1285, Accuracy: 95.52%, Epoch Time: 138.61 seconds
Epoch [3/10], Train Loss: 0.0933, Val Loss: 0.1050, Accuracy: 96.76%, Epoch Time: 138.79 seconds
Epoch [4/10], Train Loss: 0.0524, Val Loss: 0.1017, Accuracy: 96.76%, Epoch Time: 138.81 seconds
Epoch [5/10], Train Loss: 0.0275, Val Loss: 0.0751, Accuracy: 98.17%, Epoch Time: 138.61 seconds
Epoch [6/10], Train Loss: 0.0165, Val Loss: 0.0849, Accuracy: 97.84%, Epoch Time: 138.60 seconds
Epoch [7/10], Train Loss: 0.0141, Val Loss: 0.0796, Accuracy: 98.26%, Epoch Time: 138.65 seconds
Epoch [8/10], Train Loss: 0.0201, Val Loss: 0.1664, Accuracy: 96.27%, Epoch Time: 138.51 seconds
Epoch [9/10], Train Loss: 0.0180, Val Loss: 0.1656, Accuracy: 96.60%, Epoch Time: 138.47 seconds
Epoch [10/10], Train Loss: 0.0068, Val Loss: 0.0996, Accuracy: 98.42%, Epoch Time: 138.55 seconds
ViT (fine-tuned) epoch time: 

In [ ]:
# Analyse and save results
analyze_predictions(vit_ft_all_predictions, vit_ft_all_true_labels)

# Save the lists as numpy arrays for easy reloading
np.save('results/vit_ft_train_loss_list.npy', np.array(vit_ft_train_loss_list))
np.save('results/vit_ft_test_loss_list.npy', np.array(vit_ft_test_loss_list))
np.save('results/vit_ft_accuracy_list.npy', np.array(vit_ft_accuracy_list))
np.save('results/vit_ft_epoch_times_list.npy', np.array(vit_ft_epoch_times))

# Save predictions and true labels as pickle files (more efficient for storing lists of lists)
with open('results/vit_ft_all_predictions.pkl', 'wb') as f:
    pickle.dump(vit_ft_all_predictions, f)

with open('results/vit_ft_all_true_labels.pkl', 'wb') as f:
    pickle.dump(vit_ft_all_true_labels, f)

print("All fine-tuned results have been successfully saved in the 'results/' directory.")


In [ ]:
!ls 'results/'

In [ ]:
import matplotlib.pyplot as plt

# Load the results
cnn_train_loss = np.load('results/cnn_train_loss_list.npy')
cnn_test_loss = np.load('results/cnn_test_loss_list.npy')
cnn_accuracy = np.load('results/cnn_accuracy_list.npy')

cnn_res_train_loss = np.load('results/cnn_res_train_loss_list.npy')
cnn_res_test_loss = np.load('results/cnn_res_test_loss_list.npy')
cnn_res_accuracy = np.load('results/cnn_res_accuracy_list.npy')

cnn_topo_train_lost = np.load('results/cnn_topo_train_loss_list.npy')
cnn_topo_test_loss = np.load('results/cnn_topo_test_loss_list.npy')
cnn_topo_accuracy = np.load('results/cnn_topo_accuracy_list.npy')

vit_train_loss = np.load('results/vit_train_loss_list.npy')
vit_test_loss = np.load('results/vit_test_loss_list.npy')
vit_accuracy = np.load('results/vit_accuracy_list.npy')

vit_ft_train_loss = np.load('results/vit_ft_train_loss_list.npy')
vit_ft_test_loss = np.load('results/vit_ft_test_loss_list.npy')
vit_ft_accuracy = np.load('results/vit_ft_accuracy_list.npy')

# Figure 1: Plot only CNN
fig1, (ax1_1, ax2_1) = plt.subplots(1, 2, figsize=(7, 4))  # Smaller figure size
plot_accuracy_results(cnn_train_loss, cnn_test_loss, cnn_accuracy, "CNN", ax1_1, ax2_1)
plt.tight_layout()
plt.show()

# Figure 2: Plot CNN and CNN with Residual Blocks
fig2, (ax1_2, ax2_2) = plt.subplots(1, 2, figsize=(7, 4))
plot_accuracy_results(cnn_train_loss, cnn_test_loss, cnn_accuracy, "CNN", ax1_2, ax2_2)
plot_accuracy_results(cnn_res_train_loss, cnn_res_test_loss, cnn_res_accuracy, "CNN Res", ax1_2, ax2_2)
plt.tight_layout()
plt.show()

# Figure 3: Plot CNN, CNN with Residual Blocks, and CNN_Topo
fig3, (ax1_3, ax2_3) = plt.subplots(1, 2, figsize=(7, 4))
plot_accuracy_results(cnn_train_loss, cnn_test_loss, cnn_accuracy, "CNN", ax1_3, ax2_3)
plot_accuracy_results(cnn_res_train_loss, cnn_res_test_loss, cnn_res_accuracy, "CNN Res", ax1_3, ax2_3)
plot_accuracy_results(cnn_topo_train_loss, cnn_topo_test_loss, cnn_topo_accuracy, "CNN Topo", ax1_3, ax2_3)
plt.tight_layout()
plt.show()

# Figure 4: Plot CNN, CNN with Residual Blocks, CNN_Topo and ViT
fig4, (ax1_4, ax2_4) = plt.subplots(1, 2, figsize=(7, 4))
plot_accuracy_results(cnn_train_loss, cnn_test_loss, cnn_accuracy, "CNN", ax1_4, ax2_4)
plot_accuracy_results(cnn_res_train_loss, cnn_res_test_loss, cnn_res_accuracy, "CNN Res", ax1_4, ax2_4)
plot_accuracy_results(cnn_topo_train_loss, cnn_topo_test_loss, cnn_topo_accuracy, "CNN Topo", ax1_4, ax2_4)
plot_accuracy_results(vit_train_loss, vit_test_loss, vit_accuracy, "ViT", ax1_4, ax2_4)
plt.tight_layout()
plt.show()

# Figure 5: Plot CNN, CNN with Residual Blocks, CNN_Topo ViT, and Fine-tuned ViT
fig5, (ax1_5, ax2_5) = plt.subplots(1, 2, figsize=(7, 4))
plot_accuracy_results(cnn_train_loss, cnn_test_loss, cnn_accuracy, "CNN", ax1_5, ax2_5)
plot_accuracy_results(cnn_res_train_loss, cnn_res_test_loss, cnn_res_accuracy, "CNN Res", ax1_5, ax2_5)
plot_accuracy_results(cnn_topo_train_loss, cnn_topo_test_loss, cnn_topo_accuracy, "CNN Topo", ax1_5, ax2_5)
plot_accuracy_results(vit_train_loss, vit_test_loss, vit_accuracy, "ViT", ax1_5, ax2_5)
plot_accuracy_results(vit_ft_train_loss, vit_ft_test_loss, vit_ft_accuracy, "Fine-tuned ViT", ax1_5, ax2_5)
plt.tight_layout()
plt.show()


In [ ]:
# Load the epoch times for all models
cnn_epoch_times = np.load('results/cnn_epoch_times_list.npy')
cnn_res_epoch_times = np.load('results/cnn_res_epoch_times_list.npy')
cnn_topo_epoch_times = np.load('results/cnn_topo_epoch_times_list.npy')
vit_epoch_times = np.load('results/vit_epoch_times_list.npy')
vit_ft_epoch_times = np.load('results/vit_ft_epoch_times_list.npy')

# Figure 1: Plot only CNN epoch times
fig1, ax1 = plt.subplots(figsize=(7, 4))  # Smaller figure size
plot_epoch_times(cnn_epoch_times, "CNN", ax1)
plt.tight_layout()
plt.show()

# Figure 2: Plot CNN and CNN with Residual Blocks epoch times
fig2, ax2 = plt.subplots(figsize=(7, 4))
plot_epoch_times(cnn_epoch_times, "CNN", ax2)
plot_epoch_times(cnn_res_epoch_times, "CNN Res", ax2)
plt.tight_layout()
plt.show()

# Figure 3: Plot CNN, CNN with Residual Blocks, and CNN_Topo
fig3, ax3 = plt.subplots(figsize=(7, 4))
plot_epoch_times(cnn_epoch_times, "CNN", ax3)
plot_epoch_times(cnn_res_epoch_times, "CNN Res", ax3)
plot_epoch_times(cnn_topo_epoch_times, "CNN Topo", ax3)
plt.tight_layout()
plt.show()

# Figure 4: Plot CNN, CNN with Residual Blocks, CNN_Topo and ViT epoch times
fig4, ax4 = plt.subplots(figsize=(7, 4))
plot_epoch_times(cnn_epoch_times, "CNN", ax4)
plot_epoch_times(cnn_res_epoch_times, "CNN Res", ax4)
plot_epoch_times(cnn_topo_epoch_times, "CNN Topo", ax4)
plot_epoch_times(vit_epoch_times, "ViT", ax4)
plt.tight_layout()
plt.show()

# Figure 5: Plot CNN, CNN with Residual Blocks, ViT, and Fine-tuned ViT epoch times
fig5, ax5 = plt.subplots(figsize=(7, 4))
plot_epoch_times(cnn_epoch_times, "CNN", ax5)
plot_epoch_times(cnn_res_epoch_times, "CNN Res", ax5)
plot_epoch_times(cnn_topo_epoch_times, "CNN Topo", ax5)
plot_epoch_times(vit_epoch_times, "ViT", ax5)
plot_epoch_times(vit_ft_epoch_times, "Fine-tuned ViT", ax5)
plt.tight_layout()
plt.show()


In [ ]:
# If you want to download the results locally

import shutil
from google.colab import files

# Compress the 'results' folder into a zip file
shutil.make_archive('results', 'zip', 'results')

# Create a download link for the zip file
files.download('results.zip')